In [ ]:
%pylab inline

# Time-Frequency analysis with missing data

## `Stft` objects

Short-time Fourier transforms (STFT) of signals can be handled using `Stft` objects. This is a wrapper for the `ltfatpy` package. 

### Transform and inverse transform
`Stft` takes as input the parameters of the STFT, namely the hop size `hop`, the number of bins `n_bins`, the window type `win_name` and length `win_len`, as well as two other parameters, `param_constraint` (see tutorial on the constraints on the transform length) and `zero_pad_full_sig` (see tutorial on boundary effects).
   


In [ ]:
from pyteuf import Stft
from madarrays import Waveform

In [ ]:
stft = Stft(hop=16, n_bins=512, win_name='sine', win_len=256,
            param_constraint='pad', zero_pad_full_sig=False)
print(stft)

The inverse transform may obtained from the direct transform by

In [ ]:
istft = stft.get_istft()
print(istft)

### Example on a synthetic real signal

Create a signal composed of two sines

In [ ]:
nu1 = 1/33
nu2 = 3/16
duration = 0.5
fs = 8000
t = np.arange(0, int(duration*fs)) / fs
x = np.cos(2*np.pi*t*nu1*fs) + 0.5*np.cos(2*np.pi*t*nu2*fs)

Compute STFT and display the spectrogram

In [ ]:
X = stft.apply(x, fs=fs)
print(X)
_ = X.plot_spectrogram(dynrange=100.)

Note that the negative frequencies are not displayed since the STFT of a real signal is symetric hermitian, and that boundary effects appear at the beginning and the end of the time axis.

Reconstruction is obtained by:

In [ ]:
y = istft.apply(X)
print('Reconstruction: {}'.format(y))
print('Reconstruction error: {:.3f} dB'.format(10 * np.log10(np.mean(np.abs(x - y)**2))))

### Example on a synthetic complex signal

Create a waveform composed of two sines

In [ ]:
nu1 = 1/33
nu2 = 3/16
duration = 0.5
fs = 8000
t = np.arange(0, int(duration*fs)) / fs
x = Waveform(np.exp(1j*2*np.pi*t*nu1*fs) + 0.5*np.exp(1j*2*np.pi*t*nu2*fs) , fs=fs)

print(x)
np.real(x).show_player()

In [ ]:
X = stft.apply(x)
print(X)
_ = X.plot_spectrogram(dynrange=100.)

Note that all the frequencies are shown since the signal is complex.

Reconstruction is obtained by:

In [ ]:
y = istft.apply(X)
print('Reconstruction: {}'.format(y))
print('Reconstruction error: {:.3f} dB'.format(10 * np.log10(np.mean(np.abs(x - y)**2))))

### Example on a real sound

Load test sound

In [ ]:
from ltfatpy import gspi
x, fs = gspi()
x = Waveform(x, fs=fs)

Note that one may also use static method `Waveform.from_wavfile(filename)` in order to load a `Waveform` object from an audio file.

Apply STFT and display properties of Stft data

In [ ]:
X = stft.apply(x)
print(X)

Get a related Istft object, apply istft, display properties of reconstruction

In [ ]:
y = istft.apply(X)
print('Reconstruction: {}'.format(y))
print('Reconstruction error: {:.3f} dB'.format(10 * np.log10(np.mean(np.abs(x - y)**2))))

Note that `x-y` computes the difference between two `Waveform` object and returns a new `Waveform`, that may be displayed or processed, as below. Many operators can be applied in the same way to `Waveform` objects.

In [ ]:
plt.figure(figsize=(12,2))
x.plot(y_axis_label=None)

plt.figure(figsize=(12,4))
X.plot_spectrogram(dynrange=100.)

plt.figure(figsize=(12,2))
(x-y).plot(y_axis_label=None)
plt.title('Error signal')